In [ ]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../python")

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Case Study 1: Modeling

Now that the data set has been prepared, we can go to the modeling phase.

## Loading the prepared dataset



In [ ]:
from mll1 import car_pricing

In [ ]:
df_cars_raw = car_pricing.load_dataset("../../data/car_price_dataset/")
df_cars = car_pricing.prepare_dataset(df_cars_raw)

In [ ]:
df_cars.head()

In order to simplify the processing, we choose here to ignore the rows corresponding to bad quality points. In a real application, one should investigate the best approach for dealing with those points. The possible options would be to:

1. Ignore them (that is, remove those data points)
2. Leave them unchanged
3. Clip their values (since they correspond to outliers)
4. Treat them as missing values

For the last 3 possibilities, one have the additional possibility to keep or not the quality indicator columns.

Here we will simply choose the first solution:

In [ ]:
quality_columns = [c for c in df_cars.columns if c.endswith("_quality")]
mask_ok_quality = (df_cars[quality_columns] == "ok").all(1)
df_cars = df_cars[mask_ok_quality].drop(quality_columns, axis=1)
df_cars

## Determine the machine learning task

As we have stated in the introduction, when we have a real life problem at hand, we need to determine which machine learning task is the most appropriate to solve it.

In our case, the choice is quite obvious. We will perform a **regression task** since our target value is a continuous quantity.

In [ ]:
df_cars.info()

## Prediction error and performance measures

To measure the performance of a model, we need to compare the prediction to the actual values (often called ground truth).

For a regression task, since we compare continuous values, we will usually use a function related to the prediction error. For the $i$-th point, the prediction error is noted:

$$e_i = \tilde y_i - y_i$$

where we call $y_i$ and $\tilde y_i$ respectively the ground truth and the prediction for data point $i$.

Performance measures usually aggregate functions of the individual prediction errors. We shall present the most common performance measures.

### The mean squared error

$$MSE = \frac{1}{N} \sum_{i=1}^N e_i^2 = \frac{1}{N}\sum_{i=1}^N (\tilde y_i - y_i)^2$$

It is one of the most used error measures. It is also often used as the objective function to minimize in regression algorithms (hence the so called least squares).

### The root mean squared error

It it just the square root of the MSE. It has the advantage to be expressed in the same units as the $y$ values.

$$RMSE = \sqrt{\frac{1}{N}\sum_{i=1}^N e_i^2} = \sqrt{\frac{1}{N}\sum_{i=1}^N (\tilde y_i - y_i)^2}$$

### The mean absolute error

It is simply the mean of the prediction errors taken in absolute values:

$$MAE = \frac{1}{N}\sum_{i=1}^N |e_i| = \frac{1}{N}\sum_{i=1}^N |\tilde y_i - y_i|$$

It has the advantage to be directly expressed in the same units as the $y$ values and is less sensitive to extreme error values.

### The $R^2$ coefficient

$$R^2 = 1 - \frac{MSE}{Var(y)} = 1 - \frac{\sum_{i=1}^N (\tilde y_i - y_i)^2}{\sum_{i=1}^N (y_i - \bar y)^2}$$

The $R^2$ coefficient can be seen as 1 minus a normalization of the MSE by the sample variance. It has the advantage to be more or less independent of the "scale" of $y$.

A score of 1 indicates a perfect prediction while a score of $0$ indicates that we do not predict better than the mean. The score is not bounded from below.

### Explained variance

$$EV = 1 - \frac{Var(y_i - \tilde y_i)}{Var(y)} = 1 - \frac{\sum_{i=1}^N (y_i - \tilde y_i - \overline{y - \tilde y})^2 }{\sum_{i=1}^N (y_i - \bar y)^2}$$

It is very similar to the $R^2$ determination coefficient, except that the mean residual error appears as an additional term in the numerator.

Like the $R^2$, the Explained Variance Score is independent of the scale of $y$, its maximum is 1 for a perfect prediction, and it has no low bound. Both scores are equivalent when the mean residual is zero, which is the case for linear regressions.


### Mean Absolute Percentage Error

$$MAPE = \frac{1}{N}\sum_{i=1}^N \frac{|\tilde y_i - y_i|}{y_i} \times 100$$

Used for non negative targets, this is another attempt to make the score scale free. It is ill-defined when the predicted value is 0. And doesn't behave well when we need to aggregate MAPE scores computed on samples of different scales as it tends to be higher for smaller scales.

### Symmetrized MAPE

$$sMAPE = \frac{1}{N}\sum_{i=1}^N \frac{|\tilde y_i - y_i|}{\max(\tilde y_i, y_i)} \times 100$$

Individual score is counted as $0$ if $y_i = \tilde y_i = 0$.

With this definition the score is always between 0 and 100%. It still has a bias towards small values.

### Mean Squared Log Error

Another way to make a scale-free error measure is to compare the ratio  of the prediction and ground truth instead of the difference. To make the score additive, we take the log of the ratio instead:

$$MSLE = \frac{1}{N}\sum_{i=1}^N (\log(1 + \tilde y_i) - \log(1 + y_i))^2$$

This score is to be used with non-negative targets and predictions and the +1 is there to deal with zero values.

### Business-based performance measure

In the ideal case, one should always choose a performance measure that is related to the use-case of the project. In real life applications, the cost of a prediction error is often asymmetrical and exhibits threshold effects. This should be taken into account when designing the performance measure.

For instance, imagine you need to predict the number of parcels to deliver in an area and this prediction is used to allocate resources (number of drivers / trucks, number of people to sort the parcels, etc.):

* Asymmetry: While the cost of over-allocating resources could be estimated, what is the cost of not being able to deliver all the parcels in due time?
* Thresholds effects: The number of trucks to allocate is a non-smooth function of the number of parcels to deliver.

In our case study, the underestimation of the car price directly leads to a loss, while an overestimation might lead to storage costs since the car will take more time to be sold. Of course it very difficult to estimate the storage time as a function of the price.

In real life, it is indeed often quite difficult to obtain a realistic performance measure.

## Train/test split and model evaluation

Most machine learning models can be trained so that they are able to predict perfectly the target values of the training set. In most cases, this is not good news since it often means that they also generalize badly on data points that were not used for training.

To assess the generalization power of a model, it is necessary to estimate the performance on data points that have been **left out** during the training phase.

The simplest way to do this is simply to randomly remove some points from the training set and keep them in a so-called **test set**.

The `sklearn` package offers a function to do just this:

In [ ]:
from sklearn.model_selection import train_test_split

X = df_cars.drop("price", axis=1)
y = df_cars.price.values

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=0)

By default, this function randomly splits the `X` and `y` arrays in two sets. The test set size is by default 25% of the original size but can be specified as wanted through the `test_size` argument (with value between 0 and 1). Note the use of the `random_state` parameter to obtain reproducible results.

There are more sophisticated ways to perform this *data partitioning*, but we will leave that for a later chapter.

**Exercise**:
- is the row order preserved?
- how would you change that?

## Model families and selection

In this section we will test several models, and measure their performance on the test set in order to select the best performing model.  Note that there are better partitioning strategies for performing model selection (seen later in the discussion), but what follows will give the main idea.

### Linear models

Linear models are probably among of the simplest models available. Imagine we have $M$ explanatory variables $\boldsymbol x = [x_1 \dots x_M]^T$. Linear model assume that the target value $y$ is linked to the explanatory variables $X$ through:

$$ y = b + \sum_{j=1}^M x_j w_j + \varepsilon $$

Where $b$ (often called the bias or the intercept) and $\boldsymbol w = [w_1 \dots w_M]^T$ (the weight vector) need to be estimated.

The symbol $\varepsilon$ encodes the observation noise associated to each observation.

In matrix notation, the above formula is:

$$ y = b + \boldsymbol x^T \boldsymbol w + \varepsilon $$

or even:

$$\boldsymbol y = b \boldsymbol 1 + X\boldsymbol w + \boldsymbol \varepsilon$$

where $X$ is the $N \times M$ matrix containing the data points as rows:

$$\mathrm{col}_i(X) = \boldsymbol x_i^T,$$

$\boldsymbol 1$ is the $N \times 1$ dimensional vector made of all ones and $\boldsymbol \varepsilon$ is the $N \times 1$ vector containing the observation noise for each sample.

#### Least squares regression

Assume we have a set of data points represented as a set of explanatory variables and their associated target values $\{(\boldsymbol x_i, y_i)\}_{i=1}^N$.

Let $\tilde y_i = \boldsymbol x_i^T \boldsymbol w + b$ be the predicted value, we call the prediction error for the $i$-th data point the quantity:

$$e_i = \tilde y_i - y_i$$

The code below creates a data set with a single explanatory variable:

In [ ]:
def func(x):
    return 2.0 * x + 3.0


xtrain_ = np.linspace(0, 1, 11)
xtest_ = np.linspace(0, 1, 101)

np.random.seed(1)
ytrain_ = func(xtrain_) + 0.15 * np.random.randn(len(xtrain_))
ytest_ = func(xtest_)

plt.plot(xtest_, ytest_, "--", label="true function")
plt.scatter(xtrain_, ytrain_, label="observations")
plt.legend()
plt.xlabel("x")
_ = plt.ylabel("y")

Let's start with some arbitrarily chosen parameters for our model and observe the prediction errors:

In [ ]:
w = 1.0
b = 3.5

ypred_train = xtrain_ * w + b
ypred_test = xtest_ * w + b
plt.scatter(xtrain_, ytrain_, label="observations")
plt.plot(xtest_, ypred_test, c="tab:green")

pred_legend = True
for xi, yi, ypred_i in zip(xtrain_, ytrain_, ypred_train):
    plt.plot(
        [xi, xi],
        [yi, ypred_i],
        c="tab:red",
        label="prediction error" if pred_legend else None,
    )
    pred_legend = False

plt.legend()
plt.xlabel("x")
_ = plt.ylabel("y")

Our aim will be to minimize a quantity $L$ (for "loss") which aggregates all the prediction errors. Usually with a form like:

$$ L = \frac{1}{N} \sum_{i = 1}^N l(e_i) $$

where $l(.)$ is a function of the prediction error.

In least squares regression, the function $l(.)$ is the square of the prediction error :



\begin{eqnarray}
L   &=& \frac{1}{N} \sum_{i = 1}^N e_i^2 \\
L   &=& \frac{1}{N} \sum_{i = 1}^N (\tilde y_i - y_i)^2 \\
L   &=& \frac{1}{N} \sum_{i = 1}^N (\boldsymbol x_i^T \boldsymbol w + b - y_i)^2 \\
L   &=& \frac{1}{N}\|X \boldsymbol w + b \boldsymbol 1 - \boldsymbol y\|^2
\end{eqnarray}

where $\|.\|$ is the Euclidean $L_2$ norm.

Note that this corresponds exactly to the MSE score, which has the great advantage to have an analytical solution (its derivation is given in the "Going further" section at the end of notebook 3b).

The ordinary least squares regression model is implemented in the `sklearn.linear_model.LinearRegression` class:

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(xtrain_[:, np.newaxis], ytrain_)
ypred_ = reg.predict(xtest_[:, np.newaxis])

print(f"w: {reg.coef_}, b: {reg.intercept_}")

plt.plot(xtest_, ytest_, "--", label="true function")
plt.scatter(xtrain_, ytrain_, label="observations")
plt.plot(xtest_, ypred_, label="prediction")
plt.legend()
plt.xlabel("x")
_ = plt.ylabel("y")

#### Scikit-learn supervised estimators

Note the way supervised models are declared and used in `sklearn`:

We first declare the model objects with arguments if needed:

```python
model = Model(param1=value1, param2=value2)
```

The model is then fitted on the train set:

```python
model.fit(Xtrain, ytrain)
```
The `Xtrain` argument is typically a 2d numpy array of shape `(n_samples_train, n_features)` corresponding to the explanatory variables. The target values are provided through the `ytrain` argument, which, for an univariate regression as is the case here, corresponds to a 1d array of shape `(n_samples_train,)`.

Once trained, the algorithm can be applied to new data points:

```python
ypred = model.predict(Xtest)
```

The `Xtest` argument corresponds to the explanatory variables of the new points, it has a shape `(n_samples_test, n_features)` and the returned result is the prediction for those explanatory variables with shape `(n_samples_test,)`.

To measure the performance of the model, the prediction `ypred` will be compared to the the real values `ytest`.

Let's do the same thing with our data set. This dataset contains categorical columns and missing values.

To start our exploration of regression models, let's make it simpler by:

* Removing missing values (and the missing value indicator column)
* Keeping only numerical columns

In [ ]:
from sklearn.model_selection import train_test_split

num_cols = [c for c in X.columns if X.dtypes[c] == "int64" or X.dtypes[c] == "float64"]
mask_non_missing = ~df_cars["tax_mpg_missing"]

X = df_cars[mask_non_missing][num_cols]
y = df_cars[mask_non_missing].price.values

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=0)

Now let's learn our first model. In Scikit-learn, supervised models have the following API:

* `fit(X, y)`: Train the model
* `predict(X)`: Perform predictions
* `fit_predict(X, y)`: Chain train and predict steps
* `score(X, y)`: Compute a performance metric by predicting

The basic procedure will then be:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

reg = LinearRegression()
reg.fit(Xtrain, ytrain)
ypred = reg.predict(Xtest).clip(0, np.inf)  # Ensure the result is greater than 0

print(r2_score(ytest, ypred))
print(reg.score(Xtest, ytest))

**Exercise**: Can you guess why there is a difference between the two values ?

After the model is trained, some attributes are added to the `reg` object, which are the results of the learning phase. For a linear model we typically have:

* `coef_`: The `numpy` array which contains the weight vector $\boldsymbol w$ (or matrix in the case of multi-dimensional output)
* `intercept_`: The intercept (additive constant) of the linear model ($b$)

In [ ]:
reg.coef_, reg.intercept_

Scikit-learn uses the inheritance mechanism of Python for composing functionalities of the models.


Let's have a look at the `LinearRegression()` object. What are the base classes of the `LinearRegression` class ?


In [ ]:
LinearRegression.__bases__

We can see that the class has 3 parent classes:

* `LinearModel`: Base class for all linear models. It typically computes the output of the linear model $\boldsymbol w^T \boldsymbol x + b$.
* `RegressorMixin`: This does two things:
    * It tags the class as a regressor model
    * It implements the `score()` method computing the $r^2$ metric.
* `MultiOutputMixin`: This tags the model as able to deal with multi-dimensional outputs

We have prepared a function to analyze the results in the `car_pricing` module.

In [ ]:
car_pricing.plot_pred_errors(ytest, ypred)

The function does the following things:

* It computes the **mean and standard deviation of the prediction error**. This indicates if there is a bias in the prediction error. The mean value has to be compared to the standard deviation of the prediction error (which is almost equivalent to the RMSE measure).
* It computes several **performance metrics**.
* It displays the `(ytrue, ypred)` point cloud. A well behaved model should have the point cloud evenly aligned along the $ytrue = ypred$ line.

* It displays the distribution of the error prediction. A least-square based regression model should have a more or less normally distributed prediction error. We superimpose the result of a Gaussian fit and a Student-t fit (heavy tail) to illustrate the non-normality of the distribution.

**Exercise**: Comment the result.

The computation of the various scores relies mainly on the score function available in the `sklearn.metrics` submodule.

The magnitude of the weight vector of a linear model is an indication of the contribution of the corresponding variable to the result. Let's plot those values:

In [ ]:
_ = (
    pd.DataFrame({"importance": np.abs(reg.coef_)}, index=Xtrain.columns)
    .sort_values("importance")
    .plot.barh()
)

Note that the magnitude actually depends on at least two things:

* The magnitude (units) of the feature
* The magnitude of the target value.

To make the result independent on those two quantities one should:

* Standardize (center-reduce) the input values
* Divide the weights by the standard deviation of the target value

#### Pipelines

Scikit-learn offers a powerful mechanism to chain preprocessing steps and predictive models called **pipelines**.

Imagine we want to standardize the input features, we can use the `preprocessing.StandardScaler` transformer:

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)

np.mean(Xtrain_scaled, 0), np.std(Xtrain_scaled, 0)

The `StandardScaler` object is a `Transformer`:

In [ ]:
StandardScaler.__bases__

Those objects usually implement the following methods:

* `fit(X, y=None)`: to fit the transformer. It always returns `self`.
* `fit_transform(X)`: to fit the transformer and apply the transformation on the input samples.
* `transform(X)`: to apply the transformation to new points.

**Notes**:

* The `fit` method of a transformer should always accept the target as argument (even if it does not use it) in order to comply with the pipeline mechanis; hence the `y=None` optional argument.
* For the same reason, even if a transformer does not need a fit stage, it should still implement the `fit` method, and in this case simply returns the object itself.
* If you code the `transform` method, the `fit_transform` method is automatically provided by the `TransformerMixin` base class.
* Some transformers implement `fit_transform` only (and not `transform`) when they are not able to transform new points.

If we want to add this preprocessing step to our model, we should:

* fit the scaler on the training set
* transform the train set
* fit the regression model with the transformed train set
* transform the test set
* apply the predict method on the transformed test set.

The **pipeline** mechanism is there to automatize this process:

In [ ]:
from sklearn.pipeline import Pipeline

model = Pipeline([("scaler", StandardScaler()), ("reg", LinearRegression())])

model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest).clip(0, np.inf)

It is possible to chain several transformers. A pipeline is thus composed of a chain of transformers and optionally ends with a predictive model.

Imagine we want to chain some transformers and a predictive model. The different steps are given as `(name, model)` couples:

```python
models = [("tr1", transf1), ("tr2", transf2), ... ("model", model)]
```

They can be assembled in a pipeline with:

```python
pipe = Pipeline(models)
```
Then calling:
```python
pipe.fit(X, y)
```
is equivalent to doing:
```python
Xtr = X
for name, m in models[:-1]:
    Xtr = m.fit_transform(Xtr, y)

models[-1].fit(Xtr, y)
```

And calling:

```python
ypred = pipe.predict(X2)
```

is equivalent to doing:

```python
Xtr = X2
for name, m in models[:-1]:
    Xtr = m.transform(Xtr)
ypred = models[-1].predict(Xtr)
```

The steps of a pipeline can be retrieved by position through:

* The `pipe.steps` attribute which is a list of `(name, estimator)` pairs;
* The `pipe.named_steps` attribute which is a dictionary corresponding to the above pairs;
* By indexing the object either by index or by name.

In [ ]:
model.steps

In [ ]:
model.named_steps

In [ ]:
model[-1]

In [ ]:
model["reg"]

Now that we have standardized the features before fitting, let's have a look at the weight magnitude:

In [ ]:
importances = np.abs(model["reg"].coef_ / ytrain.std())  # Note the scaling
_ = (
    pd.DataFrame({"importance": importances}, index=Xtrain.columns)
    .sort_values("importance")
    .plot.barh()
)

The result is quite different and gives a better idea of the actual importance of the features.

#### Non linearities

The twisted shape of the error point cloud we saw may indicate that the price is not a linear function of the explanatory variables.

If we have some a priori business knowledge about the relationship between the target and the explanatory variables, we can include them by applying appropriate transformations to the input variables.

In the absence of prior knowledge, we can still induce some non-linearities by applying a polynomial expansion to the input variables.

For instance for a 3-variables input space with polynomial expansion of degree 2, this gives:

$$(x, y, z) \rightarrow (x, y, z, x^2, y^2, z^2, xy, xz, yz)$$


We will illustrate this on a new artificial dataset: a sinuosidal curve.
This is a one-dimensional non-linear function. Let us:

* take a few samples as training points
* add some Gaussian noise to the true function value

In [ ]:
def func(x):
    return np.sin(5.0 * x) + 3.0


np.random.seed(1)  # Fix the random number generator
xtrain_ = np.linspace(0, 1, 11)
ytrain_ = func(xtrain_) + 0.15 * np.random.randn(len(xtrain_))

xtest_ = np.linspace(0, 1, 101)
ytest_ = func(xtest_)

plt.plot(xtest_, ytest_, "--", label="true function")
plt.scatter(xtrain_, ytrain_, label="observations")
plt.xlabel("x")
plt.ylabel("y")
_ = plt.legend()

To perform the polynomial expansion, we use the `sklearn.preprocessing.PolynomialFeatures` transformer and add it to a pipeline before fitting the model:

In [ ]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

model = Pipeline(
    [
        ("scaler", StandardScaler()),
        (
            "poly",
            PolynomialFeatures(2, include_bias=False),
        ),  # The bias is redundant with the model intercept
        ("reg", LinearRegression()),
    ]
)

model.fit(xtrain_[:, np.newaxis], ytrain_)
ypred_ = model.predict(xtest_[:, np.newaxis])

plt.plot(xtest_, ytest_, "--")
plt.scatter(xtrain_, ytrain_, label="observations")
plt.plot(xtest_, ypred_, label="prediction")
print(r2_score(ypred_, ytest_))
plt.xlabel("x")
plt.ylabel("y")
_ = plt.legend()

**Exercise**: What is the best value of the polynomial degree ?

**Exercise**: Use the same pipeline on the cars dataset

Copy and paste the following code in a code cell and play with the value of `degree`.

```python

from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

poly = PolynomialFeatures(degree=?, include_bias=False)
reg = LinearRegression()

model = Pipeline([
    ("std", StandardScaler()),
    ("poly", poly),
    ("reg", reg)
])

model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest).clip(0, np.inf)

car_pricing.plot_pred_errors(ytest, ypred)

```

**Note**

`Xtrain`, `ytrain`, `Xtest`, `ytest` variables hold our train / test sets for the cars dataset.

**Warning**: The number of variables quickly explodes as the degree increases

### Regularization

When the number of variables becomes high compared to the number of data points or when explanatory variables are highly correlated, two phenomena may arise when solving the least squares problem:

* numerical instabilities
* overfitting (one can perfectly fit $N$ data points with $N-1$ variables)


#### Over-fitting
Here is an illustration:

In [ ]:
def func(x):
    return 3.0 * x + np.sin(5.0 * x) - 1.0


np.random.seed(0)
xtrain_ = np.linspace(0, 1, 11)
ytrain_ = func(xtrain_) + 0.15 * np.random.randn(len(xtrain_))

xtest_ = np.linspace(0, 1, 101)
ytest_ = func(xtest_)

plt.scatter(xtrain_, ytrain_, label="noisy observations")
plt.plot(xtest_, ytest_, "--", label="true function")
plt.xlabel("x")
plt.ylabel("y")
_ = plt.legend()

Let's fit those 11 points with a 10-th degree polynomial (10 features + 1 intercept = 11 parameters):

In [ ]:
poly = PolynomialFeatures(degree=10, include_bias=False)
reg = LinearRegression()

model = Pipeline([("poly", poly), ("reg", reg)])

model.fit(xtrain_[:, np.newaxis], ytrain_)
ypred_ = model.predict(xtest_[:, np.newaxis])

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(xtest_, ytest_, "--", label="true function")
plt.scatter(xtrain_, ytrain_, label="observations")
plt.plot(xtest_, ypred_, c="tab:orange", label="prediction")
plt.legend()
plt.xlabel("x")
plt.ylabel("y")

plt.subplot(1, 2, 2)
importances = np.abs(model["reg"].coef_) / ytrain_.std()
plt.bar(range(10), importances)
plt.xticks(range(10), ["$w^{%i}$" % (i + 1) for i in range(10)])
plt.xlabel("feature")
_ = plt.ylabel("importance")

Clearly the model above:

* **Over-fits** the training set: the prediction passes exactly through all training points
* **Poorly generalizes**: The yellow line makes large oscillations between the training points

On the right panel, we've shown the absolute values of the weight vector components. Note the "1e7" on the vertical axis. This indicates that the vector weights are very large, which is suspicious considering the actual target range. Moreover, in this kind of decomposition we more or less expect the first components to have the highest weight.

#### Detecting over-fitting

This case illustrates why it is of first importance to correctly design the model **evaluation protocol**. Evaluating the quality of the model on the same points used to train the model can lead to wrong conclusions.

The train / test split protocol allows us to easily detect the problem:

In [ ]:
from sklearn.metrics import r2_score

print(r2_score(ytrain_, model.predict(xtrain_[:, None])))
print(r2_score(ytest_, ypred_))

The code above allows us to see that:

1. The model has perfectly learned from the training dataset;
2. The model does not generalize well to new data.

Note that computing the training error can still be useful: if you do not get the performance you expected given the estimator used, that would indicate a problem in the implementation.

#### Regularized linear models: Ridge and Lasso models

To control the overfitting effect, it can be useful to add some constraints on the weight vector.
This is usually done by modifying the loss-function, which is the error that the learning algorithm tries to minimize.  One can add a term to this function, which depends on the weight-vector only:

$$ L = \frac{1}{2N}\sum_{i=1}^N (\boldsymbol x_i^T \boldsymbol w + b - y_i)^2 + \alpha \mathcal R(\boldsymbol w) $$

where $\alpha$ is a parameter setting the trade-off between the least squares term and the regularization term.

The most common regularization terms are:

* **Ridge**: $\mathcal R(\boldsymbol w) = \frac{1}{2}\|\boldsymbol w\|_2^2 = \sum_{j=1}^M w_j^2$
* **Lasso**: $\mathcal R(\boldsymbol w) = \|\boldsymbol w\|_1 = \sum_{j=1}^M |w_j|$

Let's look at what happens when we use the Ridge regression model instead of the ordinary least squares model:

In [ ]:
from sklearn.linear_model import Ridge

degree = 10

poly = PolynomialFeatures(degree=degree, include_bias=False)
reg = Ridge(alpha=1e-5)

model = Pipeline([("poly", poly), ("reg", reg)])

model.fit(xtrain_[:, np.newaxis], ytrain_)
ypred_ = model.predict(xtest_[:, np.newaxis])

print("r2 score:", r2_score(ytest_, ypred_))

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(xtest_, ytest_, "--", label="true function")
plt.scatter(xtrain_, ytrain_, label="observations")
plt.plot(xtest_, ypred_, c="tab:orange", label="prediction")
plt.legend()
plt.xlabel("x")
plt.ylabel("y")

plt.subplot(1, 2, 2)
importances = np.abs(model["reg"].coef_) / ytrain_.std()
plt.bar(range(degree), np.abs(model.named_steps["reg"].coef_))
plt.xticks(range(degree), ["$w^{%i}$" % (i + 1) for i in range(degree)])
plt.xlabel("feature")
_ = plt.ylabel("coefficient amplitude")

While keeping the same polynomial degree, we see that we can achieve good generalization. Note that the weights range is much smaller and that, as expected, low degree components have higher weight amplitudes.

**Exercise**: See how the value of the alpha parameter impacts the result

**Exercise**: Reproduce the same experiments with the `Lasso` model.

The lasso models tends to set some vector weights to 0. This is an interesting feature which implies that lasso performs some kind of **feature selection** at the same time as it fits the data points.

**Exercise**: Test with `Lasso` and `Ridge` models on the car sale dataset, and try to find the best combination of polynomial degree and `alpha` parameter.

We found the best result with the Lasso model here with a higher degree value but it is only slightly better than the ordinary least squares model in this case.

Note that Lasso can be much slower to fit than Ridge regression. Sometimes some trade off between model performance and training / prediction times must be made

### Permutation based feature importance

The use of polynomial expansion makes it difficult to use the weights vector to estimate the importance of each input feature.

**Permutation feature importance** is a model-agnostic way to determine the importance of each feature. To estimate the importance of a given feature / column, we randomly permutate its values across the rows. If the feature is important, we must observe a decrease of the model performance.

This is done for each feature possibly several times. Please refer to the scikit-learn [documentation page](https://scikit-learn.org/stable/modules/permutation_importance.html#permutation-importance) for more details.

In [ ]:
from sklearn.inspection import permutation_importance

result = permutation_importance(model, Xtest, ytest)
_ = (
    pd.DataFrame({"importance": result["importances_mean"]}, index=Xtest.columns)
    .sort_values("importance")
    .plot.barh()
)

### Nearest neighbors

Nearest neighbors models are very intuitive. To predict the value of a new data point, we find its nearest neighbors in the feature space and average the target values of the neighbors to perform the prediction.

Let's try it:

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

reg = KNeighborsRegressor()
reg.fit(Xtrain, ytrain)
ypred = reg.predict(Xtest)  # No need to clip

car_pricing.plot_pred_errors(ytest, ypred)

It seems to perform poorly.

**Exercise**:
1. Can you explain why?
1. What would you do to improve the result?

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

scaler = StandardScaler()
pca = PCA(n_components=0.9, whiten=True)
reg = KNeighborsRegressor(n_neighbors=10)

model = Pipeline([("scaler", scaler), ("pca", pca), ("reg", reg)])

model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest)  # No need to clip

car_pricing.plot_pred_errors(ytest, ypred)

The main parameter to explore here is the number of neighbors used to compute the target.

We can determine the feature importances using the permutation based algorithm:

In [ ]:
from sklearn.inspection import permutation_importance

result = permutation_importance(model, Xtest, ytest)
_ = (
    pd.DataFrame({"importance": result["importances_mean"]}, index=Xtest.columns)
    .sort_values("importance")
    .plot.barh()
)

The main inconvenients of the nearest neighbors models are:

* The full training dataset must be stored in the model;
* Finding nearest neighbors in high dimensions can become slow (use PCA ?);
* They are not well suited when there are categorical features.
* They cannot extrapolate

### Tree based models


#### Decision trees

To illustrate how decision trees work, let's first build a toy dataset with 2 variables. In the following code we build a randomly generated function defined on the unit square.

* The function is evaluated over a dense grid and the function value is represented using the pixel color
* We select some more or less regularly spread training points displayed as white circles

In [ ]:
# Create a random function on the [(0, 1), (0, 1)] unit square
# The grid_size parameter controls the 'resolution'
# of the function
f2d = car_pricing.build_2d(grid_size=4, seed=4)

# Regular grid of 2d points
r = np.linspace(0, 1, 101)
xx, yy = np.meshgrid(r, r)

# We stack the values and reshape to have a 2d matrix
# (grid_size_i * grid_size_j * n_pixels, 2)
X_ = np.dstack([xx, yy]).reshape(-1, 2)
y_ = f2d(X_)

# Selecting the train set
stride = 734
Xtrain_ = X_[stride // 2 :: stride]
ytrain_ = y_[stride // 2 :: stride]

# Draw the function value on the dense grid
# And the training point locations
cmap = "coolwarm"
plt.imshow(
    y_.reshape(101, 101),  # Reshape the (n_pixels,) array to (grid_size_i, grid_size_j)
    extent=[0, 1, 0, 1],
    origin="lower",
    cmap=cmap,
)
cmin, cmax = plt.gca().get_images()[0].get_clim()
plt.colorbar()
_ = plt.scatter(
    *Xtrain_.T, c=ytrain_, ec="k", lw=2, s=100, vmin=cmin, vmax=cmax, cmap=cmap
)

Now let's train a decision tree regressor and plot the prediction:

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

tree = DecisionTreeRegressor(min_samples_leaf=2, max_depth=None)

tree.fit(Xtrain_, ytrain_)
ypred_ = tree.predict(X_)

## Show the predicted function value
fig = plt.figure(figsize=(15, 8))
gs = fig.add_gridspec(nrows=1, ncols=2, width_ratios=[1, 2])
plt.subplot(gs[0, 0])
plt.imshow(
    ypred_.reshape(101, 101),
    extent=[0, 1, 0, 1],
    origin="lower",
    cmap=cmap,
    vmin=cmin,
    vmax=cmax,
)
plt.scatter(*Xtrain_.T, c=ytrain_, ec="k", lw=2, vmin=cmin, vmax=cmax, s=100, cmap=cmap)
plt.title("Prediction")

# Show the tree structure
plt.subplot(gs[0, 1])
plot_tree(tree)
plt.gca().set_aspect(1.0)
_ = plt.title("Tree structure")

The right panel illustrates how the tree was built. The decision tree is built recursively by choosing at each node the feature and the threshold value on this feature which optimize a criterion (MSE for regression here).

The recursive split stops when some stopping criterion is reached:

* maximum depth,
* minimum number of samples in a leaf
* ...

Once the tree is trained, we have the following properties:

* To each leaf node $l$ is associated a region $\mathcal R_l$ of the input feature space $\mathcal X$. Those regions form a partition of $\mathcal X$:

$$\mathcal X = \bigcup_{l=1}^L R_l $$
$$ \mathcal R_l \cap \mathcal R_{l'} = \emptyset\text{ if } l \neq l'$$

* The prediction of the tree is a value $w_l$ constant over the region $\mathcal R_l$ associated to a given leaf node $l$.

$$f(\boldsymbol x) = \sum_{l=1}^L w_l \mathbb I(\boldsymbol x \in \mathcal R_l)$$

* The predicted value for each leaf node $l$ is computed as an aggregation of the target values associated to training points "falling" in each leaf node region $\mathcal R_l$:

$$w_l = \frac{\sum_{i=1}^N y_i \mathbb I(\boldsymbol x_i \in \mathcal R_l)}
             {\sum_{i=1}^N \mathbb I(\boldsymbol x_i \in \mathcal R_l)}$$

where $\mathbb I(.)$ is the indicator function.

Note that this can be written as a vector product:

$$f(\boldsymbol x) = \boldsymbol r(\boldsymbol x)^T \boldsymbol w$$

where $\boldsymbol r(\boldsymbol x) = [\mathbb I(\boldsymbol x \in \mathcal R_l) \dots \mathbb I(\boldsymbol x \in \mathcal R_L)]^T$ and $\boldsymbol w = [w_1 \dots w_L]^T$

so the tree can be understood as a non-linear map $\boldsymbol r(.)$ from the input space $\mathcal X$ to a $L$-dimentional binary output space

\begin{eqnarray*}
\boldsymbol r:& \mathcal X    & \rightarrow & \{0, 1\}^L \\
  & \boldsymbol x & \mapsto & [\mathbb I(\boldsymbol x \in \mathcal R_l) \dots \mathbb I(\boldsymbol x \in \mathcal R_L)]^T
\end{eqnarray*}

and the prediction as a linear function of this mapping:

\begin{eqnarray*}
f:& \mathcal X    & \rightarrow & \mathbb R \\
  & \boldsymbol x & \mapsto & \boldsymbol r(\boldsymbol x)^T \boldsymbol w
\end{eqnarray*}


**Exercise**: Play with the `min_samples_leaf` and `max_depth` parameters and see the result

Let's go back to our dataset and apply the algorithm:

In [ ]:
from sklearn.tree import DecisionTreeRegressor

reg = DecisionTreeRegressor(min_samples_leaf=15, max_depth=None)
reg.fit(Xtrain, ytrain)

ypred = reg.predict(Xtest)  # No need to clip
car_pricing.plot_pred_errors(ytest, ypred)

The main advantages of decision trees are:

* Their **invariance to monotonic transformations** (linear or not) of the features (scaling, shifting, negation, ...). This typically saves a lot of preprocessing work.

* Their capacity to learn highly non-linear structures.
* They are easy to interpret


In the Scikit-learn implementation they do not naturally deal with categorical features, we'll see later how to process those features to make them usable with decision trees.

The main disadvantages of decision trees are:

* They tend to **easily over-fit** the training set and may lead to poor generalization.
* They may have troubles with highly correlated features
* They cannot extrapolate

Refer to the Scikit-Learn [user guide](https://scikit-learn.org/stable/modules/tree.html) for tips on how to use the decision trees.

#### Random Forests

In order to reduce the over-fitting tendency described above, Random Forests build an ensemble of decision trees by introducing 2 sources of randomness:

* **Bootstrap**: Each tree is trained on a different dataset, drawn without withdrawal from the original dataset
* **Features sub-sampling**: Each tree can be optionally trained on a randomly drawn subset of the features (less often used)

At prediction time, each tree $t$ makes a prediction $f_t(.)$ and the result of all $T$ trees are averaged to obtain the final prediction.

$$F(\boldsymbol x) = \frac{1}{T} \sum_{t=1}^T f_t(\boldsymbol x)$$

This is an example of the **bagging** (for **B**ootstrap and **agg**regat**ing**) ensemble approach.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, min_samples_leaf=2, max_depth=None)

rf.fit(Xtrain_, ytrain_)
ypred_ = rf.predict(X_)


fig = plt.figure(figsize=(15, 8))
gs = fig.add_gridspec(nrows=1, ncols=2, width_ratios=[1, 2])
plt.subplot(gs[0, 0])
plt.imshow(
    ypred_.reshape(101, 101),
    extent=[0, 1, 0, 1],
    origin="lower",
    cmap=cmap,
    vmin=cmin,
    vmax=cmax,
)
plt.scatter(*Xtrain_.T, c=ytrain_, ec="k", lw=2, vmin=cmin, vmax=cmax, s=100, cmap=cmap)
_ = plt.title("Prediction")

Let's use it on our dataset:

In [ ]:
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor(
    n_estimators=100, max_depth=15, min_samples_leaf=3, n_jobs=-1, random_state=1
)
reg.fit(Xtrain, ytrain)
ypred = reg.predict(Xtest)  # No need to clip

car_pricing.plot_pred_errors(ytest, ypred)
plt.tight_layout()
_ = (
    pd.DataFrame({"importance": reg.feature_importances_}, index=Xtest.columns)
    .sort_values("importance")
    .plot.barh()
)

We can see that the result is notably improved compared to a single decision tree.

#### Gradient Boosting

While **bagging** builds an ensemble of estimators in parallel and aggregates the individual results, **boosting** techniques consist in building iteratively a sequence of models, which try to incrementally improve the predictive performance of the previous iteration. If we call $f_i(.)$ the model built at iteration $i$, the complete model prediction is written as a weighted sum of the output of those individual models:

$$\tilde y = F(\boldsymbol x) = f_0(\boldsymbol x) + \sum_{i=1}^T \alpha_i f_i(\boldsymbol x)$$

The code below provides a **naive implementation** for the **least squares** regression case. Here, at each iteration we try to correct the prediction error:

In [ ]:
from tqdm.notebook import tqdm
from sklearn.utils import check_random_state


def train_gb(X, y, n_estimators=100, learning_rate=0.1, **tree_args):
    ypred = y.mean()  # Initial prediction

    model_info = {"y_init": y.mean(), "learning_rate": learning_rate}

    estimators = []

    # iterative improvement
    for _ in tqdm(range(n_estimators)):
        # Compute the error of the current model
        error = ypred - y
        # Fit a regression model to predict this error
        tree = DecisionTreeRegressor(**tree_args)
        tree.fit(X, error)

        # Compute the predicted error
        pred_error = tree.predict(X)
        # Partially correct the previous prediction
        ypred = ypred - learning_rate * pred_error
        # Save the correction model
        estimators.append(tree)

    model_info["estimators"] = estimators

    return model_info


def apply_gb(model_info, X):
    ypred = model_info["y_init"]
    learning_rate = model_info["learning_rate"]
    for estimator in model_info["estimators"]:
        ypred = ypred - learning_rate * estimator.predict(X)
    return ypred


tree_args = dict(max_depth=6)

model = train_gb(Xtrain, ytrain, learning_rate=0.1, **tree_args)
ypred = apply_gb(model, Xtest)
print(r2_score(ytest, ypred))

This notion of error correction can be generalized by using the gradient of the loss function. Note that in the least squares case, the gradient corresponds to the prediction error up to a multiplicative factor:

$$ \mathcal L = \sum_{i=1}^N (\tilde y_i - y_i)^2$$
$$\frac{\partial \mathcal L}{\partial \tilde y_i} = 2 (\tilde y_i - y_i) $$

**Gradient boosting** is a boosting algorithm which performs this incremental improvement by:

* fitting at each step the gradient of the loss function w.r.t to each prediction (and not w.r.t the parameters)
* using this approximation of the gradient as a descent direction to minimize the loss function.

##### Mathematical formulation*

In its most basic formulation we can express gradient boosting as the attempt to optimize a cost function:

$$\mathcal L = \frac{1}{N}\sum_{i=1}^N l(y_i, \hat y_i) $$

where $l(., .)$ is an individual loss function and $\hat y_i$ is the prediction for the $i$-th data point.

We call $F_t(\boldsymbol x) = f_0(\boldsymbol x) + \sum_{i=1}^t \alpha_t f_i(\boldsymbol x)$ the model at iteration $t$. And we call $\hat y_i^t = F_t(\boldsymbol x_i)$, the prediction for the $i$-th training point at iteration $t$.

We note the gradient of $E$ with respect to the prediction $\hat y_i^t$:

$$\frac{\partial \mathcal L}{\partial \hat y_i^t} = \frac{\partial l(y_i, \hat y_i^t)}{\partial \hat y_i^t}$$

At iteration $t+1$

1. we fit a model $f_{t+1}(.)$ on the dataset $\{(\boldsymbol x_i, \frac{\partial \mathcal L}{\partial \hat y_i^t})\}_{i=1}^N$

2. we then look for the optimal gradient descent step $\beta > 0$ (line search):

$$ \beta_{t + 1} = \min_\beta \sum_{i=1}^N l\left (y_i, F_t(\boldsymbol x_i) - \beta f_{t + 1}(\boldsymbol x_i)\right) $$

3. we build the model $F_{t+1}$ as:

$$F_{t+1}(\boldsymbol x) = F_t(\boldsymbol x) - \eta \beta_{t + 1} f_{t + 1}(\boldsymbol x)$$

where $\eta$ is a parameter between 0 and 1 usually called the **learning rate** used to *slow down* the convergence in order to improve generalization. So we have $\alpha_i = - \eta \beta_i$.

**Remarks**:

* The formulation above is valid of any model family $f(.)$. Note that even though the loss function corresponds to a classification task, the **models $f_i(.)$ are always regression models**.
* Compared to the naive implementation we described a **line search** step since, unlike for the least square case, the optimal update does not necessarily corresponds to a complete gradient update ($\beta_{t + 1} \not \approx 1$)
* The most common model family used in Gradient Boosting is the decision tree family. There are many variations based on the above idea.
* An additional bootstrap procedure can used

##### TreeBoost

When each base estimator is a decision tree some specific optimizations can be performed leading to the [TreeBoost](https://statweb.stanford.edu/~jhf/ftp/trebst.pdf) algorithm implemented in Scikit-Learn:

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor(n_estimators=100, min_samples_leaf=2, max_depth=None)

gb.fit(Xtrain_, ytrain_)
ypred_ = gb.predict(X_)


fig = plt.figure(figsize=(15, 8))
gs = fig.add_gridspec(nrows=1, ncols=2, width_ratios=[1, 2])
plt.subplot(gs[0, 0])
plt.imshow(
    ypred_.reshape(101, 101),
    extent=[0, 1, 0, 1],
    origin="lower",
    cmap=cmap,
    vmin=cmin,
    vmax=cmax,
)
plt.scatter(*Xtrain_.T, c=ytrain_, ec="k", lw=2, vmin=cmin, vmax=cmax, s=100, cmap=cmap)
_ = plt.title("Prediction")

On the car dataset, this gives:

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

reg = GradientBoostingRegressor(
    n_estimators=500, learning_rate=0.6, max_depth=4, random_state=1
)

reg.fit(Xtrain, ytrain)
ypred = reg.predict(Xtest).clip(0, np.inf)

car_pricing.plot_pred_errors(ytest, ypred)
_ = (
    pd.DataFrame({"importance": reg.feature_importances_}, index=Xtest.columns)
    .sort_values("importance")
    .plot.barh()
)

The vanilla implementation of gradient boosting in scikit-learn has the advantage of implementing some interesting loss functions (e.g. quantile loss) but it is usually quite slow. Compared to the naive implementation, the line search stage is replaced by an update of the leaf values corresponding to the (near) optimal value for the optimized loss function.

There are a variety of algorithms based on the same ideas which try to improve the performance in terms of computation speed as well as prediction performance. Among the most popular we have:

* [LightGBM](https://lightgbm.readthedocs.io/en/latest/Python-Intro.html)
* [XGBoost](https://xgboost.readthedocs.io/en/latest/python/index.html)
* [CatBoost](https://catboost.ai/) (specific treatment of categorical features)

##### HistGradientBoosting

This experimental Scikit-Learn implementation of Histogram-based Gradient Tree Boosting is based on the same ideas as `LightGBM`. It is usally much faster than the original TreeBoost algorithm:

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

reg = HistGradientBoostingRegressor(max_iter=500, min_samples_leaf=5, random_state=1)

reg.fit(Xtrain, ytrain)
ypred = reg.predict(Xtest).clip(0, np.inf)

car_pricing.plot_pred_errors(ytest, ypred)
result = permutation_importance(reg, Xtest, ytest)
_ = (
    pd.DataFrame({"importance": result["importances_mean"]}, index=Xtest.columns)
    .sort_values("importance")
    .plot.barh()
)

##### XGBoost

In [XGBoost](https://xgboost.ai) (eXtreme Gradient Boosting), the loss function is taken into account while building the tree structure at each iteration.

In [ ]:
from xgboost import XGBRegressor

reg = XGBRegressor(n_estimators=300, max_depth=5, learning_rate=0.5, random_state=1)

reg.fit(Xtrain, ytrain)
ypred = reg.predict(Xtest).clip(0, np.inf)

car_pricing.plot_pred_errors(ytest, ypred)
_ = (
    pd.DataFrame({"importance": reg.feature_importances_}, index=Xtest.columns)
    .sort_values("importance")
    .plot.barh()
)

##### CatBoost

This implementation performs categorical feature encoding as a part of the process and implements some specific tricks in order to avoid over-fitting.

Let's try it first without categorical features, so we can compare it to the previous algorithms:

In [ ]:
from catboost import CatBoostRegressor

reg = CatBoostRegressor(
    n_estimators=700, learning_rate=0.5, depth=6, random_state=1, verbose=False
)

reg.fit(Xtrain, ytrain)
ypred = reg.predict(Xtest).clip(0, np.inf)

car_pricing.plot_pred_errors(ytest, ypred)
_ = (
    pd.DataFrame({"importance": reg.feature_importances_}, index=Xtest.columns)
    .sort_values("importance")
    .plot.barh()
)

Scikit-learn proposes a small guide to choose the appropriate algorithm for a given dataset

https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

## Next steps

We have seen a number of linear and tree-based model families, and how to measure their performance.

In the following notebook, we will introduce techniques for:

- dealing with categorical variables;
- handling missing values;
- perform model selection and hyper-parameters optimization;
- save and reload pre-trained models.